##### Copyright 2020 Google Inc.

Licensed under the Apache License, Version 2.0 (the "License").
<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->


# TFDV and Apache Beam sandbox

In [ ]:
%pip install tfx

In [13]:
import logging
import re

import pyarrow as pa

from typing import List, Optional, Text, Union, Dict, Iterable

import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.runners import DataflowRunner

from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

from apache_beam.io.gcp.internal.clients import bigquery

#from tensorflow_data_validation import types

from jinja2 import Template

import google.auth

In [2]:
%%bigquery
select * from `covertype_dataset.covertype`
limit 100

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2067,0,21,270,9,755,184,196,145,900,Cache,C2702,5
1,2574,0,2,319,20,1419,216,235,156,1595,Commanche,C2703,4
2,2559,0,0,510,16,1113,218,238,156,1332,Commanche,C2703,2
3,2647,0,6,402,94,641,212,229,155,1104,Commanche,C2703,2
4,2651,0,3,335,103,488,215,233,156,1381,Commanche,C2703,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2985,0,8,470,66,4747,208,225,154,6225,Rawah,C4744,1
96,2718,0,7,162,28,2300,210,227,155,4541,Rawah,C4744,1
97,2724,0,2,255,41,2434,216,234,156,4986,Rawah,C4744,1
98,2722,0,12,404,33,1931,203,218,152,6014,Rawah,C4744,1


In [3]:
class DataDriftOptions(PipelineOptions):
    @classmethod
    def _add_argparse_args(cls, parser):
        parser.add_argument(
        '--baseline_data_location',
        help="Input for the pipeline",
        default='gs://workshop-datasets/covertype/small')
        
        parser.add_argument(
        '--target_data_table',
        help="A target BQ table with data to analyze",
        default="covertype_dataset.covertype")
        
   

In [4]:
def generate_sampling_query(source_table_name, num_lots, lots):
  """Prepares the data sampling query."""

  sampling_query_template = """
       SELECT *
       FROM 
           `{{ source_table }}` AS cover
       WHERE 
       MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), {{ num_lots }}) IN ({{ lots }})
       """
  query = Template(sampling_query_template).render(
      source_table=source_table_name, num_lots=num_lots, lots=str(lots)[1:-1])

  return query


In [5]:
source_table_name = 'mlops-dev-env.covertype_dataset.covertype'
num_lots = 100
lots = [1]

query = generate_sampling_query(source_table_name, num_lots, lots)

In [6]:
options = PipelineOptions()
options.view_as(GoogleCloudOptions).project = 'mlops-dev-env'

In [7]:
custom_options = options.view_as(DataDriftOptions)

In [8]:
#p = beam.Pipeline(options=options)

In [9]:
p = beam.Pipeline(InteractiveRunner(), options=options)

In [42]:
class BatchedDictsToRecordBatch(beam.DoFn):
    """DoFn to convert a batch of dictionaries to a pyarrow.RecordBatch.
  
    In the current implementation, the function relies on automatic
    coercion to pyarrow data types. 
    To be verified whether this is a right approach.
    """

    def process(self, batch: List[Dict]) -> Iterable[pa.RecordBatch]:
        
        column_names = batch[0].keys()
        values_by_column = {column_name: [] for column_name in column_names}
        for row in batch:
            for key, value in row.items():
                values_by_column[key].append(value)
                
        arrow_arrays = [
            pa.array(arr) for arr in values_by_column.values()
            ]  
        
        yield pa.RecordBatch.from_arrays(arrow_arrays, list(column_names))
  

In [44]:
#@beam.typehints.with_input_types(types.BeamCSVRecord)
#@beam.typehints.with_output_types(pa.RecordBatch)
class DecodeBigQuery(beam.PTransform):
    """Decodes BigQuery records into Arrow RecordBatches."""
    def __init__(
        self,
        column_names: List):
    
        if not isinstance(column_names, list):
            raise TypeError('column_names is of type %s, should be a list' %
                            type(column_names).__name__)
        self._column_names = column_names
    
    def expand(self, pcoll):
        record_batches = (
            pcoll
            | beam.Map(lambda row: {field: row[field]
                                    for field in self._column_names})
            | beam.BatchElements(min_batch_size=2, max_batch_size=3)
            | beam.ParDo(
                BatchedDictsToRecordBatch())
            )

        return record_batches

In [45]:
records = (p
           | 'query' >> beam.io.Read(beam.io.BigQuerySource(query=query, use_standard_sql=True))
           | 'batch' >> DecodeBigQuery(['Elevation', 'Aspect'])
          )

In [46]:
ib.show_graph(p)

In [47]:
ib.show(records)

/root/apache-beam-custom/packages/beam/sdks/python/apache_beam/transforms/util.py:384: RuntimeWarning: divide by zero encountered in true_divide
  cook_ds = 0.5 / s2 * errs**2 * (h / (1 - h)**2)
/root/apache-beam-custom/packages/beam/sdks/python/apache_beam/transforms/util.py:423: RankWarning: Polyfit may be poorly conditioned
  a, b = self.linear_regression(xs, ys)
/root/apache-beam-custom/packages/beam/sdks/python/apache_beam/transforms/util.py:423: RankWarning: Polyfit may be poorly conditioned
  a, b = self.linear_regression(xs, ys)
/root/apache-beam-custom/packages/beam/sdks/python/apache_beam/transforms/util.py:423: RankWarning: Polyfit may be poorly conditioned
  a, b = self.linear_regression(xs, ys)
/root/apache-beam-custom/packages/beam/sdks/python/apache_beam/transforms/util.py:423: RankWarning: Polyfit may be poorly conditioned
  a, b = self.linear_regression(xs, ys)
/root/apache-beam-custom/packages/beam/sdks/python/apache_beam/transforms/util.py:384: RuntimeWarning: invali

In [ ]:
words = (p 
         | 'read' >> beam.io.ReadFromText(custom_options.data_location)
         | 'extract' >> beam.FlatMap(lambda line: re.findall(r'[\w\']+', line.strip(), re.UNICODE))
)


In [ ]:
ib.show_graph(p)

In [ ]:
counts = (words 
          | 'count' >> beam.combiners.Count.PerElement())

In [ ]:
ib.show_graph(counts)

In [ ]:
d1 = {'a': 2, 'b': 'hello'}

In [ ]:
list(d1.keys())

In [ ]:
d1.values()